Everytime you edit any cell click run on that cell

# user inputs: 

In [52]:
ticker = "JNJ" # Uppercase stock ticker in the S&P 500

start_date = '01-01-2018' # Enter start date in format 'MM-DD-YYYY'
end_date = '01-01-2019' # Enter end date in format 'MM-DD-YYYY'

enable_RSI = False # True to filter based on RSI condition, False !!!!!
RSI_period = 14
RSI_conditional_value = 40  # [overbought, oversold]: To be considered a falling knife RSI has to be outside of these two values

enable_MACD = False
MACD_short_term_period = 12
MACD_long_term_period = 26
MACD_conditional_value = -.2

enable_return = True
RETURN_conditional_value = -.05

graph_closing_price = False # True to graph, False to not graph

# user should only click run on the cells below.  DO NOT EDIT CELLS BELOW UNLESS YOU KNOW WHAT YOU ARE EDITING

You only have to run this first cell once

In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as style
import matplotlib.dates as mdates
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import seaborn as sns
sns.set_style("darkgrid")
from ta.momentum import RSIIndicator
from ta.trend import MACD
from datetime import datetime, timedelta
SP_500_database = pd.read_csv("database2.csv")

Run this next cell every time you edit some thing in the user input cell

In [54]:
# cleans aznd reformats the database based on the ticker
ticker_database = SP_500_database[SP_500_database["Ticker"]==ticker]
ticker_database = ticker_database.round(2)
ticker_database['Date'] = pd.to_datetime(ticker_database['Date'])
ticker_database = ticker_database.set_index('Date')

#print(ticker_database.head(-5))
#filters database based on your choosen start and end date
#print(ticker_database.head(-5))

# creates indicator data
ticker_database['MACD'] = MACD(ticker_database["Adj Close"], n_slow=MACD_short_term_period, n_fast=MACD_long_term_period).macd()
ticker_database['RSI'] = RSIIndicator(ticker_database["Adj Close"], RSI_period).rsi()
ticker_database['Return'] = ticker_database["Adj Close"].pct_change()

ticker_database_filtered = ticker_database

if enable_RSI:
    ticker_database_filtered = ticker_database_filtered[ticker_database_filtered['RSI'] < RSI_conditional_value]

if enable_MACD:
    ticker_database_filtered = ticker_database_filtered[ticker_database_filtered["MACD"] <= MACD_conditional_value]

if enable_return:
    ticker_database_filtered = ticker_database_filtered[ticker_database_filtered["Return"] <= RETURN_conditional_value]

# fallingKniveDf = df[df['Adj Close'] < df['SMA']]

print("There are " + str(len(ticker_database_filtered)) + " falling knives")
print(ticker_database_filtered)


There are 6 falling knives
           Ticker            Company    Open   Close  Adj Close    Return  \
Date                                                                        
2008-10-09    JNJ  Johnson & Johnson   62.64   57.58      40.62 -0.076608   
2008-10-15    JNJ  Johnson & Johnson   64.31   60.54      42.71 -0.054042   
2008-12-01    JNJ  Johnson & Johnson   57.66   55.33      39.36 -0.055436   
2018-02-05    JNJ  Johnson & Johnson  136.58  130.39     122.68 -0.052957   
2018-12-14    JNJ  Johnson & Johnson  144.34  133.00     128.53 -0.100434   
2019-10-18    JNJ  Johnson & Johnson  133.31  127.70     126.02 -0.062212   

              High     Low      Volume  GICS Sector GICS Sub Industry  \
Date                                                                    
2008-10-09   63.16   57.58  33493900.0  Health Care   Pharmaceuticals   
2008-10-15   64.76   60.10  25064700.0  Health Care   Pharmaceuticals   
2008-12-01   57.82   55.22  18847200.0  Health Care   Pharmaceut

This next cell manages the graphing

In [55]:
# Create figure
fig = make_subplots(rows=3, cols=1, shared_xaxes=True)

fig.add_trace(go.Scatter(x=ticker_database.index, y=ticker_database['Adj Close']), row=3, col=1)

fig.add_trace(go.Scatter(x=ticker_database_filtered.index, y=ticker_database_filtered['Adj Close'],
                    mode='markers',
                    name='falling knives'),
                    row=3, col=1)

fig.add_trace(go.Scatter(x=ticker_database.index, y=ticker_database['MACD']), row=1, col=1)

fig.add_trace(go.Scatter(x=ticker_database_filtered.index, y=ticker_database_filtered['MACD'],
                    mode='markers',
                    name='falling knives'),
                    row=1, col=1)

fig.add_trace(go.Scatter(x=ticker_database.index, y=ticker_database['RSI']), row=2, col=1)

fig.add_trace(go.Scatter(x=ticker_database_filtered.index, y=ticker_database_filtered['RSI'],
                    mode='markers',
                    name='falling knives'),
                    row=2, col=1)


# Add range slider
fig.update_layout(
    title_text="Finding Falling Knives",
    xaxis3_rangeslider_visible=True,
    height=800,
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        type="date"
    )
)

fig.show()